## Load modules

In [1]:
push!(LOAD_PATH, "../dvi")

addprocs(int(CPU_CORES / 2))

using DiscreteValueIteration
using PilotSCAs, PilotSCAIterators, PilotSCAConst
using GridInterpolations

mdp = SCA()

SCA(5446,36,StateIterator([-2000.0,-1600.0,-1200.0,-800.0,-400.0,0.0,400.0,800.0,1200.0,1600.0,2000.0],[-2000.0,-1600.0,-1200.0,-800.0,-400.0,0.0,400.0,800.0,1200.0,1600.0,2000.0],[0.0,1.5708,3.14159,4.71239,6.28319],[10.0,15.0,20.0],[10.0,15.0,20.0],1,1,1,1,1,5446),ActionIterator([:right20,:right10,:straight,:left10,:left20,:clearOfConflict],1,1,36),RectangleGrid with 5445 points)

## Check size of MDP

In [2]:
function getBytes(x)
   total = 0;
   fieldNames = typeof(x).names;
   if fieldNames == ()
      return sizeof(x);
   else
     for fieldName in fieldNames
        total += getBytes(getfield(x,fieldName));
     end
     return total;
   end
end

println("mdp of type ", typeof(mdp), " takes up ", getBytes(mdp) / 1000.0, " kB")

mdp of type SCA takes up 1.768 kB


## Informal validation of state and action iterators and transition function

In [3]:
testActions = ActionIterator(Actions)
actionCount = 0
for action in testActions
    actionCount += 1
    if actionCount == 1 || actionCount == NActions
        println("action #", actionCount, ": ", typeof(action), " = ", action)
    end # if
end # for action
println("nActions = ", NActions)
println("actionCount = ", actionCount, "\n")

testStates = StateIterator(Xs, Ys, Bearings, Speeds, Speeds)
stateCount = 0
for state in testStates
    stateCount += 1
    if stateCount == 1 || stateCount == NStates
        println("state #", stateCount, ": ", typeof(state), " = ", state)
    end # if
end # for state
println("nStates = ", NStates)
println("stateCount = ", stateCount)

action #1: Action = Action(:right20,:right20)
action #36: Action = Action(:clearOfConflict,:clearOfConflict)
nActions = 36
actionCount = 36

state #1: State = State(-2000.0,-2000.0,0.0,10.0,10.0,false)
state #5446: State = State(-2000.0,-2000.0,0.0,10.0,10.0,true)
nStates = 5446
stateCount = 5446


In [4]:
nextStateIndices, probs = nextStates(mdp, 1, 15)
println("next state indices:\n", nextStateIndices, "\n")
println("probabilities:\n", probs, "\n")
println("probabilities sum to ", sum(probs))

next state indices:
[1,1,2,5446,5446,1,2,1816,1817,5446,5446,5446,5446]

probabilities:
[0.3333333333333333,0.08125,0.0020833333333333355,0.08333333333333334,0.08333333333333334,0.04875,0.0012500000000000013,0.03250000000000001,0.0008333333333333341,0.08333333333333334,0.08333333333333334,0.08333333333333334,0.08333333333333334]

probabilities sum to 1.0


## Parallel solution

In [5]:
numProcs = int(CPU_CORES / 2)
solver = ParallelSolver(
    numProcs,
    maxIterations = 1000,
    tolerance = 1e-2,
    gaussSiedel = true,
    includeV = true,
    includeQ = true,
    includeA = true)

ParallelSolver(2,None[],1000,0.01,true,true,true,true)

In [ ]:
policy = solve(solver, mdp, verbose = true)
println("")

Iter 1: runtime = 1.414e+01, net runtime = 1.414e+01
Iter 2: runtime = 1.431e+01, net runtime = 2.845e+01
Iter 3: runtime = 1.239e+01, net runtime = 4.084e+01
Iter 4: runtime = 1.302e+01, net runtime = 5.386e+01
Iter 5

In [ ]:
function sharray2array(sharray::SharedArray{Float64, 2})
    result = zeros(sharray.dims)
    for i = 1:sharray.dims[1]
        for j = 1:sharray.dims[2]
            result[i, j] = sharray[i, j]
        end # for j
    end # for i
    return result
end # function sharray2array

In [ ]:
solQ = sharray2array(policy.Q')
save("../../data/pilot-alpha.jld", "solQ", solQ)

## Check against reference solution by visual inspection

In [ ]:
push!(LOAD_PATH, "../src/")
using Pairwise, Multiagent, JLD

d = get_pomdp()
refQ = load("../../data/alpha.jld", "alpha")
solQ = load("../../data/pilot-alpha.jld", "solQ")
println("")

### Reference policy plot

In [ ]:
viz_pairwise_policy(d, refQ)

### Parallel solver policy plot

In [ ]:
viz_pairwise_policy(d, solQ)